# Taller Visión por Computador Reconocimiento del Lenguaje de Señas

## <font color= green> Primera Parte </font>

## Deteccion de Puntos de Referencia Manuales

### Deteccion de Puntos de referencia en una imagen estatica

In [14]:
## Importamos las dependencias
import cv2 as cv
import mediapipe as mp
import matplotlib.pyplot as plt
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

#### Opciones de configuración de Parametros

- **STATIC_IMAGE_MODE:** Si se establece en `false`, la solución trata las imágenes de entrada como una transmisión de video. Intentará detectar las manos en las primeras imágenes de entrada y, tras una detección exitosa, localiza aún más los puntos de referencia manuales. En imágenes posteriores, una vez todas max_num_hands se detectan las manos y se localizan los puntos de referencia de la mano correspondiente, simplemente rastrea esos puntos de referencia sin invocar otra detección hasta que pierde el rastro de cualquiera de las manos. Esto reduce la latencia y es ideal para procesar cuadros de video. Si se establece en `true`, la detección manual se ejecuta en cada imagen de entrada, ideal para procesar un lote de imágenes estáticas, posiblemente no relacionadas. Predeterminado a `false`.

- **MAX_NUM_HANDS:** Número máximo de manos para detectar. Predeterminado a `2.`

- **MODEL_COMPLEXITY:** Complejidad del modelo de punto de referencia manual: `0 o 1`. La precisión del marcador, así como la latencia de inferencia, generalmente aumentan con la complejidad del modelo. Predeterminado a `1`.

- **MIN_DETECTION_CONFIDENCE:** Valor mínimo de confianza `([0.0, 1.0])` del modelo de detección manual para que la detección se considere exitosa. Predeterminado a 0.5.

- **MIN_TRACKING_CONFIDENCE:** Valor mínimo de confianza `([0.0, 1.0])` del modelo de seguimiento de puntos de referencia para que los puntos de referencia manuales se consideren rastreados con éxito, o de lo contrario la detección manual se invocará automáticamente en la siguiente imagen de entrada. Establecerlo en un valor más alto puede aumentar la robustez de la solución, a expensas de una latencia más alta. Ignorado si `static_image_mode` es true, donde la detección manual simplemente se ejecuta en cada imagen. Predeterminado a `0.5.`

In [2]:
# declaramos la ruta de la imagen a trabajar

# configuramos parametos de deteccion



A continuacion cargamos la imagen utilizando opencv, realizamos un cambio de espacio de color, y graficamos la imagen

### Mostrando los resultados

El modelo realiza la deteccion de 21 puntos importantes representados como cordenadas 3d (x,y,z), de las uniones o nudillos dentro de la region de la mano, a continuacion se muestra los 21 puntos que nos retorna el modelo.

![imagen](./hand_landmarks.png)

### Salida
El estilo de denominación puede diferir ligeramente entre plataformas / idiomas.

- **MULTI_HAND_LANDMARKS:** Colección de manos detectadas / rastreadas, donde cada mano se representa como una lista de 21 puntos de referencia manuales y cada punto de referencia se compone de ` x, y y z`. `x y y` se normalizan a `[0.0, 1.0]` por el ancho y la altura de la imagen respectivamente. `z` representa la profundidad de referencia con la profundidad en la muñeca como origen, y cuanto menor sea el valor, más cerca estará el punto de referencia de la cámara. La magnitud de `z` usa aproximadamente la misma escala que `x`.

- **MULTI_HAND_WORLD_LANDMARKS:** Colección de manos detectadas / rastreadas, donde cada mano se representa como una lista de `21 puntos de referencia manuales en coordenadas mundiales`. Cada punto de referencia se compone de `x, y y z`: coordenadas 3D del mundo real en metros con el origen en el centro geométrico aproximado de la mano.

- **MULTI_HANDEDNESS:** Recolección de la mano de las manos detectadas / rastreadas ( i.e. ¿Es una mano izquierda o derecha ). Cada mano está compuesta de label y score. label es una cadena de valor "Left" o "Right". score es la probabilidad estimada de la entrega prevista y siempre es mayor o igual que `0.5` ( y la mano opuesta tiene una probabilidad estimada de `1 - score)`.

Tenga en cuenta que la capacidad de entrega se determina suponiendo que la imagen de entrada esté reflejada, es decir, tomada con una cámara frontal / selfie con imágenes volteadas horizontalmente. Si no es así, intercambie la salida de entrega en la aplicación.

**Prediccion de Inferncias**

Ha continuación haciendo uso de la función `process()` la cual Procesa una imagen RGB y devuelve los puntos de referencia de la mano obtenemos los `hand landmarks` de las manos detectadas en la imagen.

### Mostrando configuración de la salida

#### Obteniendo la precision e identificando la mano

#### Obteniendo los marcadores

#### Obteniendo cada punto en Particular

![imagen](./hand_landmarks.png)

### Dibujando y Mostrando los Puntos y sus conexiones

### Dibujando las cajas delimitadoras

### Dibujando y mostrando los puntos y sus cajas delimitadoras

### Dibujando y mostrando los puntos, cajas delimitadoras, mano correspondiente y precisión

## Deteccion de Puntos de Referencia en Video

### Programa Final

In [1]:
## Importacion de librerias
import cv2 as cv
import mediapipe as mp

## referenciando metodos necesarios
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# configurando parametros de deteccion
hands = mp_hands.Hands(max_num_hands=2,
                       min_detection_confidence=0.8, 
                       min_tracking_confidence=0.8
                      )
# creacion de ventanas para mostrar los resultados
cv.namedWindow("MediaPipe Hands", cv.WINDOW_NORMAL)

## *****************************************************************************
## Funcion para dibujar los puntos y sus conexiones
def hand_detect(img):
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    result = hands.process(img)
    
    if result.multi_hand_landmarks: 
        
        img = draw_box(result, img)
        
        for hand_landmarks in result.multi_hand_landmarks:
            mp_drawing.draw_landmarks(img, 
                                      hand_landmarks, 
                                      mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(),
                                      mp_drawing_styles.get_default_hand_connections_style()
                                      )
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    
    return img
## *****************************************************************************
## Funcion para dibujar los cuadros delimitadores
def draw_box(result, img):
    
    hand_id, hand_score = score(result)
    cont =0
    for hand_landmarks in result.multi_hand_landmarks:
        xlist=[]
        ylist=[]
        for id, lm in enumerate(hand_landmarks.landmark):
            h, w, c = img.shape
            cx, cy = int(lm.x * w), int(lm.y * h)
            xlist.append(cx)
            ylist.append(cy)

        xmin, xmax = min(xlist), max(xlist)
        ymin, ymax = min(ylist), max(ylist)
        
        cv.rectangle(img, (xmin-20, ymin-20), (xmax+20, ymax+20),(0, 255, 0), 2)
        text = str(hand_id[cont])+" "+str(hand_score[cont])+"%"
        cv.putText(img, 
                    text, 
                    (xmin-20, ymin-20), 
                    cv.FONT_HERSHEY_SIMPLEX, 
                    0.5, 
                    (255,0,0),2)
        cont+=1
        
    return img

## *****************************************************************************
## Funcion para encontrar la Mano correspondiente y su porcentaje de presicion en la deteccion
def score(result):
    hand_id=[]
    hand_score = []
    for hand in result.multi_handedness:
        
        for id, lm in enumerate(hand.classification):
            hand_id.append(lm.label)
            hand_score.append(int(lm.score*100))
            
    return hand_id, hand_score
               
## *****************************************************************************
## ----------- CODIGO PRINCIPAL--------------------------
## creacion del Objeto captura de video 
cap = cv.VideoCapture(0)

while cap.isOpened():
    
    ret, frame = cap.read() 
    
    if ret:
        frame = cv.flip(frame, 1)
        img_proces = frame.copy()
        image = hand_detect(img_proces)
        cv.imshow("MediaPipe Hands", image)
        
        key = cv.waitKey(10)
        if key == 27:
            break
        
    else:
        break
            
cap.release()
cv.destroyAllWindows()

## Proyecto Capturar la imagen de la mano para crear datasets